# Autoencoder training for Latent Space Visualization
The autoencoder is trained on the FFHQ dataset with 16 dimensions and 128x128 image size.

In [ ]:
import tensorflow.keras.layers
import tensorflow.keras.models
import tensorflow.keras.optimizers
import tensorflow.keras.datasets
import numpy as np
import matplotlib.pyplot as plt

## Encoder

In [ ]:
x = tensorflow.keras.layers.Input(shape=(128*128), name="encoder_input")

encoder_dense_layer1 = tensorflow.keras.layers.Dense(units=2048, name="encoder_dense_1")(x)
encoder_activ_layer1 = tensorflow.keras.layers.LeakyReLU(name="encoder_leakyrelu_1")(encoder_dense_layer1)

encoder_dense_layer2 = tensorflow.keras.layers.Dense(units=1024, name="encoder_dense_1")(x)
encoder_activ_layer2 = tensorflow.keras.layers.LeakyReLU(name="encoder_leakyrelu_1")(encoder_dense_layer2)

encoder_dense_layer3 = tensorflow.keras.layers.Dense(units=512, name="encoder_dense_1")(x)
encoder_activ_layer3 = tensorflow.keras.layers.LeakyReLU(name="encoder_leakyrelu_1")(encoder_dense_layer3)

encoder_dense_layer4 = tensorflow.keras.layers.Dense(units=16, name="encoder_dense_2")(encoder_activ_layer3)
encoder_output = tensorflow.keras.layers.LeakyReLU(name="encoder_output")(encoder_dense_layer4)

encoder = tensorflow.keras.models.Model(x, encoder_output, name="encoder_model")
encoder.summary()

## Decoder

In [ ]:
decoder_input = tensorflow.keras.layers.Input(shape=(16), name="decoder_input")

decoder_dense_layer1 = tensorflow.keras.layers.Dense(units=128, name="decoder_dense_1")(decoder_input)
decoder_activ_layer1 = tensorflow.keras.layers.LeakyReLU(name="decoder_leakyrelu_1")(decoder_dense_layer1)

decoder_dense_layer2 = tensorflow.keras.layers.Dense(units=256, name="decoder_dense_1")(decoder_input)
decoder_activ_layer2 = tensorflow.keras.layers.LeakyReLU(name="decoder_leakyrelu_1")(decoder_dense_layer2)

decoder_dense_layer3 = tensorflow.keras.layers.Dense(units=1024, name="decoder_dense_1")(decoder_input)
decoder_activ_layer3 = tensorflow.keras.layers.LeakyReLU(name="decoder_leakyrelu_1")(decoder_dense_layer3)

decoder_dense_layer4 = tensorflow.keras.layers.Dense(units=2048, name="decoder_dense_1")(decoder_input)
decoder_activ_layer4 = tensorflow.keras.layers.LeakyReLU(name="decoder_leakyrelu_1")(decoder_dense_layer4)

decoder_dense_layer5 = tensorflow.keras.layers.Dense(units=128*128, name="decoder_dense_2")(decoder_activ_layer4)
decoder_output = tensorflow.keras.layers.LeakyReLU(name="decoder_output")(decoder_dense_layer5)

decoder = tensorflow.keras.models.Model(decoder_input, decoder_output, name="decoder_model")
decoder.summary()

Join the two model into an autoencoder.

In [ ]:
ae_input = tensorflow.keras.layers.Input(shape=(128*128), name="AE_input")
ae_encoder_output = encoder(ae_input)
ae_decoder_output = decoder(ae_encoder_output)

ae = tensorflow.keras.models.Model(ae_input, ae_decoder_output, name="AE")
ae.summary()

### Root mean square deviation

In [ ]:
def rmse(y_true, y_predict):
    return tensorflow.keras.backend.mean(tensorflow.keras.backend.square(y_true-y_predict))

## Load the dataset

In [1]:
path = "../dataset/"
data = []

from PIL import Image
import os
from tqdm import tqdm

for img in tqdm(os.listdir(path)[:7000]):
    data.append(np.array(Image.open(path+img).resize((128, 128))))
data = np.average(np.array(data).astype("float32") / 255.0, axis=3)

x_train = np.reshape(data[:6000], newshape=(data[:6000].shape[0], np.prod(data[:6000].shape[1:])))
x_test = np.reshape(data[1000:], newshape=(data[1000:].shape[0], np.prod(data[1000:].shape[1:])))

del data

## Compile the Autoencoder

In [ ]:
ae.compile(loss="mse", optimizer=tensorflow.keras.optimizers.Adam(learning_rate=0.0005))

## Train the Autoencoder

In [ ]:
ae.fit(x_train, x_train, epochs=1000, batch_size=256, shuffle=True, validation_data=(x_test, x_test))

encoded_images = encoder.predict(x_train)
decoded_images = decoder.predict(encoded_images)
decoded_images_orig = np.reshape(decoded_images, newshape=(decoded_images.shape[0], 128, 128))

num_images_to_show = 5
for im_ind in range(num_images_to_show):
    plot_ind = im_ind*2 + 1
    rand_ind = np.random.randint(low=0, high=x_train.shape[0])
    plt.subplot(num_images_to_show, 2, plot_ind)
    plt.imshow(x_train[rand_ind].reshape((128, 128)), cmap="gray")
    plt.subplot(num_images_to_show, 2, plot_ind+1)
    plt.imshow(decoded_images_orig[rand_ind, :, :], cmap="gray")

fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot()
scatter = ax.scatter(encoded_images[:, 0], encoded_images[:, 1])#, c=y_train)
plt.colorbar(scatter)

# Save the model

In [ ]:
ae.save("ae.h5")

encoder.save("encoder.h5")
decoder.save("decoder.h5")